## Problem Statement

The case study is from an open source dataset from Kaggle. 

Link to the Kaggle project site:

https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling (Links to an external site.)Links to an external site.

Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?


The points distribution for this case is as follows:

Read the dataset
1. Drop the columns which are unique for all users like IDs (2.5 points)

2. Distinguish the feature and target set (2.5 points)

3. Divide the data set into Train and test sets

4. Normalize the train and test data (2.5 points)

5. Initialize & build the model (10 points)

6. Optimize the model (5 points)

7. Predict the results using 0.5 as a threshold (5 points)

8. Print the Accuracy score and confusion matrix (2.5 points)

## Solution

In [89]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt


In [90]:
#Importing the dataset
bank_df = pd.read_csv('bank.csv')
bank_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [91]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [92]:
bank_df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [93]:
# Removing features from the dataset which seems to be same for all users and few unwanted columns
bank_df = bank_df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [94]:
# one-hot encoding for object datatype
bank_df = pd.get_dummies(bank_df)

In [95]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null int64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Geography_France     10000 non-null uint8
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
Gender_Female        10000 non-null uint8
Gender_Male          10000 non-null uint8
dtypes: float64(2), int64(7), uint8(5)
memory usage: 752.0 KB


In [96]:
# Dividing the dataset into features and target
features = bank_df.drop("Exited", axis=1)
target = bank_df["Exited"]

In [97]:
# Splitting the dataset into the Training and Testing set.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.25, random_state = 0)

In [98]:
#Normalizing the train and test data 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [99]:
# importing tensorflow packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation

print(tf.__version__)

1.13.1


In [112]:
#empty model
model = Sequential()

#input / first hidden layer
model.add(Dense(units = 13, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13))

#second hidden layer
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

#output layer
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [113]:
# output shape
model.output_shape

# summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_31 (Dense)             (None, 6)                 84        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 7         
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [114]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=["accuracy"])

In [115]:
model.fit(X_train, y_train, batch_size = 10, epochs = 20)

Epoch 1/20
7500/7500 [==============================] - 3s 351us/sample - loss: 0.5000 - acc: 0.7960
Epoch 2/20
7500/7500 [==============================] - 2s 299us/sample - loss: 0.4216 - acc: 0.8185
Epoch 3/20
7500/7500 [==============================] - 2s 297us/sample - loss: 0.4125 - acc: 0.8288
Epoch 4/20
7500/7500 [==============================] - 2s 306us/sample - loss: 0.4081 - acc: 0.8304
Epoch 5/20
7500/7500 [==============================] - ETA: 0s - loss: 0.4028 - acc: 0.833 - 2s 318us/sample - loss: 0.4044 - acc: 0.8325
Epoch 6/20
7500/7500 [==============================] - 3s 343us/sample - loss: 0.4003 - acc: 0.8349
Epoch 7/20
7500/7500 [==============================] - 2s 316us/sample - loss: 0.3932 - acc: 0.8345
Epoch 8/20
7500/7500 [==============================] - 3s 346us/sample - loss: 0.3850 - acc: 0.8384
Epoch 9/20
7500/7500 [==============================] - 3s 357us/sample - loss: 0.3788 - acc: 0.8415
Epoch 10/20
7500/7500 [==============================

In [116]:
#Predicting the Test set results
y_test_pred = model.predict(X_test)

#Predicting the results using 0.5 as a threshold
y_test_pred = (y_test_pred > 0.5)

y_test_pred

array([[False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [

In [117]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_pred)

array([[1872,  119],
       [ 232,  277]], dtype=int64)

In [118]:
from sklearn.metrics import accuracy_score

print("Test accuracy is ",(accuracy_score(y_test_pred, y_test)*100),'%')

Test accuracy is  85.96000000000001 %
